# The 2-variable polynomial invariant of knotted 4-valent graphs

This notebook is intended to show the process of finding the multiset of 2-variable polynomial invariants for a choice of knotted 4-valent graph $G$, and a knot invariant polynomial. These polynomials are defined in "Knotted 4-regular graphs: polynomial invariants and the Pachner moves" ([arXiv:2206.05816](https://arxiv.org/abs/2206.05816)). It uses Python code from the package Graphpoly to do this; here are the steps used, given a specific Dowker-Thistletwaite sequence for a knotted graph.

1. Find the orientation of the graph nodes by running the DT algorithm on the sequence, giving the $f(i)$ function on node labels $i$. The convention is that $f(i) = +1$ if the other arc crosses from right to left, and $f(i) = -1$ if the other arc crosses from left to right.
1. Find the planar diagram (PD) code for the graph, using the DT sequence and the orientation just found.
1. From the PD code, find all possible Eulerian circuits through the graph. This gives the multiset of induced knot diagrams for the knot.
1. Choose a link polynomial $L(K)$ which is invariant for any diagram of the link $K$. Use the skein relation given below for the vertices of the graph to find a polynomial in the variables $c, s$, with coefficients given by link diagrams. The brackets with subscript '$L$' refer to evaluating the resulting link using the link polynomial $L(K)$.
1. For the work given below, the link polynomial $L(K)$ is chosen to be the Jones polynomial in terms of the variable $A$. This is done as shown by Adams in *The Knot Book*, Section 6.2.

![](./img/eqns.png)

For this notebook, we will use the example of the graph given by the DT sequence $3^\ell 5^+ 7^- 1^u$. This is represented as a Python list by

```python
    seq = [[0, 3, -2], [2, 5, 1], [4, 7, -1], [6, 1, 2]]
```

For each node, there is a list of three numbers in this list, denoting an Eulerian circuit through the graph. The first element is an even number, the second an odd; these are the two labels for that particular node. The node labels are in the set $\{0, 1, \cdots, 2N - 1\}$, for a graph with $N$ nodes. The third element gives the crossing or vertex type: $-2$ for a vertex with the upper edge on the odd label (label superscript $\ell$), $-1$ for a crossing with the upper edge on the odd label (label superscript $-$), and the positive numbers are the flipped cases (superscripts $u$ and $+$). When the node is a vertex, the "upper crossing" means the line drawn on the graph diagram; see Section III of the paper "Knotted 4-regular graphs".

## Find the orientation of the graph nodes

First, we import the function `isRealizable(seq, f_list = False)`. If the function is given a Python list `seq`, the function returns `True` if the sequence is a realizable DT sequence, and `False` if the sequence does not represent a planar graph. Note that the given sequence `seq` may or may not include the crossing or vertex type. If the parameter `f_list` is set to `True`, for a realizable sequence, the function additionally will return a list of $f(i)$ values for each node label $i$. These indicate the crossing information at the node $i$ for the Eulerian circuit described by the DT sequence. See "Classification of knot projections", Topology Appl. 16, 19–31 (1983), for details.

In [1]:
from Graphpoly.isRealizable import isRealizable

The next command creates the orientation list `orientList` for the example DT sequence `seq`. Remember that DT sequences do not differentiate between a sequence and its mirror image. Thus, one can make a global change in sign of the $f(i)$ for all $i$, and still have a realizable sequence.

In [2]:
# Example sequence 3^l 5^+ 7^- 1^u

seq = [[0, 3, -2], [2, 5, 1], [4, 7, -1], [6, 1, 2]]

orientList = isRealizable(seq, f_list = True)
orientList

[1, 1, -1, -1, 1, 1, -1, -1]

## Find the planar diagram code

The function `planarDiagram(seq, f_list)` takes the DT sequence `seq` and the orientation list `f_list`, and produces a planar diagram list from them. Thus, the function returns a list of two lists. The first list gives the PD code for the graph. **Note:** the numbers in this list refer to *half-edges* (or *darts*), not *nodes*. There can be problems if an induced knot diagram has self-loops, so using half-edges allows for a unique number associated with each of the four half-edges incident to the node.

A vertex is treated as the appropriate crossing, based on its vertex state. The second list gives the node type for each node. Again, we use 1 for a crossing, and 2 for a vertex. The sign of the numbers comes from $f(i)$, so they are positive if the upper crossing is right-to-left, and negative if left-to-right.

In [3]:
from Graphpoly.planarDiagram import planarDiagram

Here, we show the planar diagram notation for $3^\ell 5^+ 7^- 1^u$, where the orientation is that found above in a previous cell. The node type, along with the value of $f(i)$, is recorded in `nodeTypeList`. Starting from the DT sequence given above, the darts incident to each node are labeled as follows. Recall each node has an even label $\ell_e$ and an odd label $\ell_o$ from the Eulerian circuit. Thus, we can label the darts using the labels $2 \ell_e, 2 \ell_e + 1, 2 \ell_o, 2 \ell_o + 1$, where the first two correspond to the incoming and outgoing darts for the edges associated with the even DT sequence label, and the last two for the odd sequence label. This will give a unique number for each dart in the range $\{0, 4N - 1\}$.

In [4]:
[PD_code, nodeTypeList] = planarDiagram(seq, orientList)

print('PD code =', PD_code)
print('node type list =', nodeTypeList)

PD code = [[0, 6, 1, 7], [2, 12, 3, 13], [8, 14, 9, 15], [10, 4, 11, 5]]
node type list = [2, 2, 1, 1]


## List all induced knot diagrams

We use a modified version of Hierholzer's algorithm to find all possible Eulerian circuits through the graph. Note that there is only one possible way to pass along the circuit when coming into a crossing, but three possible ways for a vertex -- one choice for each of the other three incident edges. Thus, the algorithm must take this freedom into account, using the list `nodeTypeList` obtained from `planarDiagram()`. The algorithm is implemented in `createCircuits()`.

In [5]:
from Graphpoly.createCircuits import createCircuits

The function `createCircuits()` returns a list of `graph` objects, each of which represents a particular choice of Eulerian circuit through the original graph. The half-edges of the circuit can be returned using `graph.listEPath()`. The labels of the half-edges are the same as those used in `planarDiagram()`, as described in the last section.

In [6]:
Ecircuits = createCircuits(PD_code, nodeTypeList)
graphList = [graph for graph in Ecircuits]

for graph in graphList:
    print('ePath:', graph.listEPath())
    print('PD code:', graph.listPDCode())
    print('node type dict:', graph.listNodeTypeDict())
    print('---')
    
print('Number of circuits:', len(graphList))

ePath: [0, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
PD code: [[1, 7, 0, 6], [3, 13, 2, 12], [9, 15, 8, 14], [11, 5, 10, 4]]
node type dict: {0: 2, 1: 2, 2: 1, 3: 1}
---
ePath: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
PD code: [[0, 6, 1, 7], [2, 12, 3, 13], [8, 14, 9, 15], [10, 4, 11, 5]]
node type dict: {0: 2, 1: 2, 2: 1, 3: 1}
---
ePath: [0, 15, 14, 13, 12, 11, 10, 9, 8, 7, 1, 2, 3, 4, 5, 6]
PD code: [[1, 7, 0, 6], [2, 12, 3, 13], [9, 15, 8, 14], [11, 5, 10, 4]]
node type dict: {0: 2, 1: 2, 2: 1, 3: 1}
---
ePath: [0, 6, 5, 4, 3, 2, 1, 7, 8, 9, 10, 11, 12, 13, 14, 15]
PD code: [[0, 6, 1, 7], [3, 13, 2, 12], [8, 14, 9, 15], [10, 4, 11, 5]]
node type dict: {0: 2, 1: 2, 2: 1, 3: 1}
---
ePath: [0, 15, 14, 13, 2, 1, 6, 5, 4, 3, 12, 11, 10, 9, 8, 7]
PD code: [[1, 7, 0, 6], [3, 13, 2, 12], [9, 15, 8, 14], [11, 5, 10, 4]]
node type dict: {0: 2, 1: 2, 2: 1, 3: 1}
---
ePath: [0, 7, 8, 9, 10, 11, 12, 3, 4, 5, 6, 1, 2, 13, 14, 15]
PD code: [[0, 6, 1, 7], [2, 12, 3, 13], [8,

## Finding the 2-variable polynomial

There is now enough information to create the 2-variable graph polynomial invariant, by using the induced knot diagrams just obtained with the original PD code for the knotted graph. This will allow us to figure out how each Eulerian circuit passes through any given vertex, and therefore which of the three skein relations given above to use for that vertex. Going through this process for each vertex, we can construct the $c$ and $s$ terms of the polynomial for a particular Eulerian circuit.

To proceed, we must introduce the two variables used in the polynomial, along with the variable $A$ used in the Jones polynomial.

In [7]:
var('c, s, A')

(c, s, A)

### Link components

In general, using the skein relations will create links from the induced knot diagrams. This is because, depending on the connections at each vertex, the number of link components can increase or decrease by one for the $s$ terms. Keeping track of this is a crucial part of the algorithm, and is done by having multiple sublists in the list storing information about the Eulerian circuit. At the beginning, there is only one list, since the induced knot diagram is one continuous path through the entire graph. However, for the first $s$ term, the number of components will increase to two; then, there will be two sublists inside of the overall `e_path` list. Each sublist will have only the darts on that link component.

As an example, we consider the vertex shown below. Before the split, the circuit passes through the vertex, and the rest of the graph, in the order $a \to c \to \cdots \to b \to d \to \cdots \to a$.

![](./img/split-01.png)

After the $s$ split, there are two components, the $a \to d \to \cdots \to a$ piece, and the $b \to c \to \cdots \to b$ part. Thus, the half-edges of these two link components are now kept in two sublists in the overall connection information. This means we have the original dart list `[[ac ... bd]]` becomes the list `[[ad ...], [bc ...]]`. Note how the connections have now changed in the final link -- for example, $d$ follows $a$ now, instead of $b$. For situations like this, we use the function `splitSegment()` to take the original segment, and divide it in two, with the proper connection information afterwards.

In [8]:
from Graphpoly.splitSegment import splitSegment

In another case, suppose we have the vertex given in the picture below. Now before the split, there were two disconnected segments to the circuit; the $s$ operation in the skein relation combines them back together.

![](./img/split-02.png)

Specifically, in the initial case, we have $a \to c \to \cdots \to a$, and $b \to d \to \cdots \to b$. Afterwards, the circuit has the connections $a \to d \to \cdots b \to c \to \cdots \to a$. Thus, the list `e_path` goes from `[[ac ...], [bd ...]]` to `[[ad ... bc ...]`. This process is done with the function `sewSegments()`.

In [9]:
from Graphpoly.sewSegments import sewSegments

There are twelve such cases overall to consider -- for each of the three skein relations, there are the choices of whether the circuit through the vertex is connected or not, and the orientation of the upper edge. However, all involve each using `sewSegments()` to decrease the number of segments by one, or `splitSegment()` to increase the number by one.

We now import two additional helper functions to assist in keeping track of this information. The function `segIndexList()` takes the `e_path` for the induced graph, and a specific node, and returns the location of each dart incident to the node in `e_path`. Specifically, for each dart, a tuple `(seg, index)` is returned. The first value `seg` gives which list in`e_path` the dart is located in; the second value `index` gives where the dart is located within the given list. The function `connectList()` returns this list, along with a second list. This list gives the connections between the darts at the node as two tuples. For example, for the node with PD code `[d0, d1, d2, d3]`, the list `[(d0, d1), (d3, d2)]` shows that the induced knot diagram follows the path $d_0 \to d_1$ and $d_3 \to d_2$.

In [10]:
from Graphpoly.connectList import connectList
from Graphpoly.segIndexList import segIndexList

Here is an example of the two functions in use. The `e_path` given has two separate link components, and we focus on a particular node.

In [11]:
ePath = [[11, 10, 9, 8, 7, 0, 15, 14, 13, 12], [4, 3, 2, 1, 6, 5]]
node = [1, 0, 7, 6]

segList = segIndexList(ePath, node)
print(segList)

connect = connectList(ePath, node)
print(connect)

[(1, 3), (0, 5), (0, 4), (1, 4)]
[[(1, 3), (0, 5), (0, 4), (1, 4)], [(1, 6), (7, 0)]]


### Jones polynomial using states

The above gives us a way to keep track of the link components formed after each application of the skein relations to a vertex. After all of this is done, however, we need a way to find the Jones polynomial invariant for the resulting link that is the coefficient of a particular $c^m s^n$ term in the 2-variable polynomial. As mentioned at the beginning, we use the technique given in Section 6.2 of Adams' *The Knot Book*. This is implemented in the function `LPoly()`.

In [12]:
from Graphpoly.LPoly import LPoly

The function `LPoly` goes through the crossing in the list `e_path`, and finds the writhe number for the link. Then, it progressively applies the skein relation for links, eventually splitting the link down to $N$ unknots. This choice is a *state* for the link. Each state is reported as a tuple `(A, B, N)`, where $A$ is the number of splittings giving a multiplicative factor of $A$, while $B$ gives the number of $A^{-1}$, and $N$ is the number of unknots. These can be put back together in Sage to give an overall Jones polynomial for the original link.

Here are some test cases, consisting of links with small numbers of crossings and components.

In [13]:
# Unknot w/no crossings

#ePath = [[0, 1]]
#PD_code = []

# Unknot w/one crossing

#ePath = [[0, 1, 2, 3]]
#PD_code = [[0, 2, 1, 3]]

# Two unknots w/no crossings

#ePath = [[0, 1], [2, 3]]
#PD_code = []

# Two unknots, one w/crossing

#ePath = [[10, 9, 3, 4], [7, 8, 2, 1, 0, 11, 5, 6]]
#PD_code = [[1, 7, 0, 6]]

# Hopf link (+1)

#ePath = [[0, 1, 2, 3], [4, 5, 6, 7]]
#PD_code = [[0, 4, 1, 5], [6, 2, 7, 3]]

# Hopf link (+1) + unknot

#ePath = [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9]]
#PD_code = [[0, 4, 1, 5], [6, 2, 7, 3]]

# Hopf link (-1)

#ePath = [[0, 1, 2, 3], [4, 7, 6, 5]]
#PD_code = [[0, 4, 1, 5], [7, 3, 6, 2]]

# Hopf link (-1) + unknot

#ePath = [[0, 1, 2, 3], [4, 7, 6, 5], [8, 9]]
#PD_code = [[0, 4, 1, 5], [7, 3, 6, 2]]

# Trefoil knot (example in Adams, p. 158)

#ePath = [list(range(12))]
#PD_code = [[0, 6, 1, 7], [8, 2, 9, 3], [4, 10, 5, 11]]

# Trefoil (mirror image)

#ePath = [list(range(12))]
#PD_code = [[6, 1, 7, 0], [2, 9, 3, 8], [10, 5, 11, 4]]

# Figure eight knot

ePath = [list(range(16))]
PD_code = [[0, 6, 1, 7], [12, 3, 13, 2], [4, 11, 5, 10], [8, 14, 9, 15]]

# Results (eqn 6.1 in Adams)

[w, stateList] = LPoly(ePath, PD_code)
func = 0
for state in stateList:
    func += A ** state[0] * A ** (-state[1]) * (-A ** 2 - A ** -2) ** (state[2] - 1)
func *= (-A ** 3) ** -w

print(func.expand())

A^8 - A^4 - 1/A^4 + 1/A^8 + 1


### Assembling the 2-variable polynomial

The final piece is the code that uses all of the above as a Python script in Sage. This function is called `twoVarPoly()`. This function takes a list `e_path` for a particular Eulerian circuit of a knotted graph, the PD code `PD_code` for all nodes in the graph, and the type list `type_list` for these nodes. For this choice of circuit, it returns a polynomial in the variables $c, s$ (from the vertices) and $A, A^{-1}$ (from the link coefficients). The code currently uses recursive calls -- unlike `LPoly()` -- but could be modified to use a priority queue if this speeds things up.

In [14]:
def twoVarPoly(e_path, PD_code, type_list):
    
    # If the graph has been completely reduced down to a link,
    # compute the Jones polynomial by (1) finding the writhe
    # number, and (2) the bracket polynomial. Combining them
    # gives the Jones polynomial as a function of A = t^-4.
    
    if type_list == []:
        
        # Compute information for Jones polynomial
        
        [w, stateList] = LPoly([dart for dart in e_path], \
                              [node for node in PD_code])
        
        # Build up final polynomial using this information
        
        func = 0
        for (a, b, S) in stateList:
            func += (A ** a) * (A ** -b) * (-A ** 2 - A ** -2) ** (S - 1)
            
        return func * (-A ** 3) ** -w
    
    # If the graph has not be completely reduced, process
    # the next node. To keep track of which node we are
    # considering next, we delete the node type for the
    # node after the processing is done, since we do not
    # use it to find the graph polynomial. This is because
    # nodes may not be deleted from the PD code if they are
    # crossings (or converted into them).
    
    next_node_index = len(type_list) - 1
    current_node = PD_code[next_node_index]
    
    # Find the segment and connection lists for current e_path, node
    
    [seg_list, connect_list] = connectList([label for label in e_path], \
                                           [label for label in current_node])
    
    # If the node type is a crossing, check to make sure that
    # PD code for current node matches orientation of e_path,
    # then recursively find the invariant graph polynomial.
    
    if abs(type_list[-1]) == 1:
        
        # Create new PD_code
        
        new_PD_code = [node for node in PD_code]
        
        # Check whether the given node is correct for the
        # circuit through the graph, by seeing if the first
        # dart listed comes before the third listed. Note that
        # the first and third darts must be on the same
        # component, even if the original single circuit has
        # be subdivided.
        
        if (current_node[2], current_node[0]) in connect_list:
            new_PD_code[next_node_index] = [[current_node[2], current_node[3], \
                                            current_node[0], current_node[1]]]
            
        # Return resulting graph polynomial
        
        return twoVarPoly([[dart for dart in segment] for segment in e_path], \
                         new_PD_code, [type for type in type_list[:-1]])
    
    # If the node type is a vertex, use the graph polynomial
    # skein relation. This will depend on how the Eulerian circuit
    # passes through the vertex, so we will have to check a number
    # of cases to see how the relation is applied to the vertex.
    
    elif abs(type_list[-1]) == 2:
        
        # Give darts simple names
        
        [d0, d1, d2, d3] = current_node
        
        # First check is whether the vertex is proper or not. If
        # the vertex is proper, the c component keeps the current
        # node (as a crossing), along with the same e_path. NOTE:
        # we are assuming that the undercrossing edge does not
        # go through the vertex as d2 -> d0!
        
        if (d0, d2) in connect_list:
        
            c_func = twoVarPoly([[dart for dart in segment] for segment in e_path], \
                                [node for node in PD_code], [type for type in type_list[:-1]])
            
            # To determine the s component, we see in what
            # direction the overcrossing edge passes through
            # the vertex, along with whether the edges are in
            # the same component of the induced graph diagram.
            
            if seg_list[0][0] == seg_list[1][0]:
                
                # The two edges are in the same segment, so
                # the s move will break e_path into two segments.
                
                if (d1, d3) in connect_list:                       # Right-to-left overcrossing
                    
                    new_e_path = splitSegment([seg for seg in e_path], seg_list[0][0], \
                                             seg_list[2][1], seg_list[1][1], seg_list[3][1], \
                                             seg_list[0][1])
                    
                else:                                              # Left-to-right overcrossing
                    
                    new_e_path = splitSegment([seg for seg in e_path], seg_list[0][0], \
                                             seg_list[2][1], seg_list[3][1], seg_list[1][1], \
                                             seg_list[0][1])
            else:
            
                # The two edges are in different segments, so
                # the s move will join them together into
                
                if (d1, d3) in connect_list:
                    
                    new_e_path = sewSegments([seg for seg in e_path], seg_list[0][0], seg_list[1][0], \
                                            seg_list[2][1], seg_list[0][1], seg_list[3][1], seg_list[1][1])
                    
                elif (d3, d1) in connect_list:
                    
                    new_e_path = sewSegments([seg for seg in e_path], seg_list[0][0], seg_list[1][0], \
                                            seg_list[2][1], seg_list[0][1], seg_list[1][1], seg_list[3][1])
                        
            # Assemble everything together for s coefficient of vertex
                    
            s_func = twoVarPoly(new_e_path, [node for node in PD_code if node != current_node], \
                                [type for type in type_list[:-1]])
            
            return (c * c_func + s * s_func)
        
        # Vertex is not proper. In this case, the c component
        # removes the current vertex, but keeps the same e_path.
        
        else:
            
            c_func = twoVarPoly([[dart for dart in segment] for segment in e_path], \
                                [node for node in PD_code if node != current_node], \
                                [type for type in type_list[:-1]])
                
            # Now we determine whether the two connections through the
            # vertex are in the same segment or not. If they are in the
            # same segment, we break the segment in two; otherwise, we
            # sew the two segments together into one. Since we know that
            # d0, d2 are *not* connected, use these two darts to see the
            # number of segments.
            
            if seg_list[0][0] == seg_list[2][0]:                # In same segment
                if (d0, d1) in connect_list:
                    if (d2, d3) in connect_list:                # d0 -> d1, d2 -> d3; del node
                        
                        new_e_path = splitSegment([seg for seg in e_path], seg_list[0][0], \
                                                 seg_list[1][1], seg_list[2][1], seg_list[3][1], \
                                                 seg_list[0][1])
                        new_PD_code = [node for node in PD_code if node != current_node]
                        
                    elif (d3, d2) in connect_list:              # d0 -> d1, d3 -> d2; keep node
                        
                        new_e_path = splitSegment([seg for seg in e_path], seg_list[0][0], \
                                                 seg_list[1][1], seg_list[3][1], seg_list[2][1], \
                                                 seg_list[0][1])
                        new_PD_code = [node for node in PD_code]
                        
                elif (d0, d3) in connect_list:
                    if (d1, d2) in connect_list:                # d0 -> d3, d1 -> d2; keep node
                        
                        new_e_path = splitSegment([seg for seg in e_path], seg_list[0][0], \
                                                 seg_list[3][1], seg_list[1][1], seg_list[2][1], \
                                                 seg_list[0][1])
                        new_PD_code = [node for node in PD_code]
                        
                    elif (d2, d1) in connect_list:              # d0 -> d3, d2 -> d1; del node
                        
                        new_e_path = splitSegment([seg for seg in e_path], seg_list[0][0], \
                                                 seg_list[3][1], seg_list[2][1], seg_list[1][1], \
                                                 seg_list[0][1])
                        new_PD_code = [node for node in PD_code if node != current_node]
                        
                elif (d1, d0) in connect_list and (d3, d2) in connect_list:
                    
                    # d1 -> d0, d3 -> d2; del node
                    
                    new_e_path = splitSegment([seg for seg in e_path], seg_list[0][0], \
                                             seg_list[0][1], seg_list[3][1], seg_list[2][1], \
                                             seg_list[1][1])
                    new_PD_code = [node for node in PD_code if node != current_node]
                    
                elif (d1, d2) in connect_list and (d3, d0) in connect_list:
                    
                    # d1 -> d2, d3 -> d0; del node
                    
                    new_e_path = splitSegment([seg for seg in e_path], seg_list[0][0], \
                                             seg_list[0][1], seg_list[1][1], seg_list[2][1], \
                                             seg_list[3][1])
                    new_PD_code = [node for node in PD_code if node != current_node]
                        
            else:                                               # In different segments
                if (d0, d1) in connect_list:
                    if (d2, d3) in connect_list:                # d0 -> d1, d2 -> d3; del node
                        
                        new_e_path = sewSegments([seg for seg in e_path], seg_list[0][0], seg_list[2][0], \
                                                seg_list[1][1], seg_list[0][1], seg_list[3][1], seg_list[2][1])
                        new_PD_code = [node for node in PD_code if node != current_node]
                        
                    elif (d3, d2) in connect_list:              # d0 -> d1, d3 -> d2; keep node
                        
                        new_e_path = sewSegments([seg for seg in e_path], seg_list[0][0], seg_list[2][0], \
                                                seg_list[1][1], seg_list[0][1], seg_list[2][1], seg_list[3][1])
                        new_PD_code = [node for node in PD_code]
                        
                elif (d0, d3) in connect_list:
                    if (d1, d2) in connect_list:                # d0 -> d3, d1 -> d2; keep node
                        
                        new_e_path = sewSegments([seg for seg in e_path], seg_list[0][0], seg_list[2][0], \
                                                seg_list[3][1], seg_list[0][1], seg_list[2][1], seg_list[1][1])
                        new_PD_code = [node for node in PD_code]
                        
                    elif (d2, d1) in connect_list:              # d0 -> d3, d2 -> d1; del node
                        
                        new_e_path = sewSegments([seg for seg in e_path], seg_list[0][0], seg_list[2][0], \
                                                seg_list[3][1], seg_list[0][1], seg_list[1][1], seg_list[2][1])
                        new_PD_code = [node for node in PD_code if node != current_node]
                        
                elif (d1, d0) in connect_list and (d3, d2) in connect_list:
                        
                    # d1 -> d0, d3 -> d2; del node
                        
                    new_e_path = sewSegments([seg for seg in e_path], seg_list[0][0], seg_list[2][0], \
                                                seg_list[0][1], seg_list[1][1], seg_list[2][1], seg_list[3][1])
                    new_PD_code = [node for node in PD_code if node != current_node]
                        
                elif (d1, d2) in connect_list and (d3, d0) in connect_list:
                        
                    # d3 -> d0, d1 -> d2; del node
                        
                    new_e_path = sewSegments([seg for seg in e_path], seg_list[0][0], seg_list[2][0], \
                                                seg_list[0][1], seg_list[3][1], seg_list[2][1], seg_list[1][1])
                    new_PD_code = [node for node in PD_code if node != current_node]
            
            # Assemble everything together for s coefficient of vertex
            
            s_func = twoVarPoly(new_e_path, new_PD_code, [type for type in type_list[:-1]])
            
            return (c * c_func + s * s_func)

Now that this procedure has been developed, we can run through all the Eulerian circuits for the given graph, finding the corresponding 2-variable polynomial invariant. These are placed in the list `polyList`.

In [15]:
polyList = []
for graph in graphList:
    polyList += [twoVarPoly([graph.listEPath()], graph.listPDCode(), \
                            list(graph.listNodeTypeDict().values()))]

Printing out the raw list of polynomials is usually a mess.

In [16]:
polyList

[((A^4 - 4*(A^2 + 1/A^2)*A^2 + 2*(A^2 + 1/A^2)^2 - 4*(A^2 + 1/A^2)/A^2 + (A^2 + 1/A^2)^2/A^4 + 4)*A^12*c + ((A^2 + 1/A^2)*A^3 - (A^2 + 1/A^2)^2*A - 2*A + 3*(A^2 + 1/A^2)/A - (A^2 + 1/A^2)^2/A^3)*A^9*s)*c + (((A^2 + 1/A^2)*A^3 - (A^2 + 1/A^2)^2*A - 2*A + 3*(A^2 + 1/A^2)/A - (A^2 + 1/A^2)^2/A^3)*A^9*c - (A^2 - (A^2 + 1/A^2)^2/A^2 + 2/A^2)*A^6*s)*s,
 ((A^4 - 4*(A^2 + 1/A^2)*A^2 + 2*(A^2 + 1/A^2)^2 - 4*(A^2 + 1/A^2)/A^2 + (A^2 + 1/A^2)^2/A^4 + 4)*A^12*c + ((A^2 + 1/A^2)*A^3 - (A^2 + 1/A^2)^2*A - 2*A + 3*(A^2 + 1/A^2)/A - (A^2 + 1/A^2)^2/A^3)*A^9*s)*c + (((A^2 + 1/A^2)*A^3 - (A^2 + 1/A^2)^2*A - 2*A + 3*(A^2 + 1/A^2)/A - (A^2 + 1/A^2)^2/A^3)*A^9*c - (A^2 - (A^2 + 1/A^2)^2/A^2 + 2/A^2)*A^6*s)*s,
 -((A^2 + 1/A^2)*c + ((A^2 + 1/A^2)*A^2 - (A^2 + 1/A^2)^2 + (A^2 + 1/A^2)/A^2 - 1)*s)*s - c*((A^3 - 3*(A^2 + 1/A^2)*A + (A^2 + 1/A^2)^2/A + 2/A - (A^2 + 1/A^2)/A^3)*c/A^3 - ((A^2 + 1/A^2)*A^3 - (A^2 + 1/A^2)^2*A - 2*A + 3*(A^2 + 1/A^2)/A - (A^2 + 1/A^2)^2/A^3)*s/A^3),
 -((A^2 + 1/A^2)*c + ((A^2 + 1/A^

It is more helpful to place these in a dictionary, to better reflect the multiset character of the information. The keys are the polynomials themselves, and the values are the number of times each polynomial appears. Then, we can print this out.

In [17]:
polyDict = {}
for poly in polyList:
    polyDict[poly] = polyDict.get(poly, 0) + 1
    
for poly, num in polyDict.items():
    print(num, ':', poly.coefficients(c))       # Give coefficient of each c term as separate list
    #print(num, ':', poly.expand())             # Expand out in c^m s^n terms

2 : [[s^2, 0], [-2*A^10*s - 2*A^2*s, 1], [-A^16 + A^12 + A^4, 2]]
2 : [[s^2, 0], [-A^2*s - 2*s/A^2 - s/A^10, 1], [1, 2]]
2 : [[-A^16*s^2 + A^12*s^2 + A^4*s^2, 0], [-2*A^10*s - 2*A^2*s, 1], [1, 2]]
2 : [[s^2, 0], [-A^2*s - 2*s/A^2 - s/A^10, 1], [1, 2]]
2 : [[s^2, 0], [-A^2*s - 2*s/A^2 - s/A^10, 1], [1, 2]]
2 : [[s^2, 0], [-A^2*s - 2*s/A^2 - s/A^10, 1], [1, 2]]
